In [16]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from collections import defaultdict
from pprint import pprint
from datetime import datetime
from textblob import TextBlob

import gensim
import os
import re
#author: https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
#modified: ranjanmanish
tokenizer = RegexpTokenizer(r'\w+')

dictionary = []
dictFile = open("dict.txt")
for word in dictFile:
    word = word.strip()
    dictionary.append(word.lower())

negList = open("negative-words.txt")

for negWord in negList:
    negWord = negWord.strip()
    dictionary.append(negWord.lower())

foulList = open("foulWordList.txt")

for foulWord in foulList:
    foulWord = foulWord.split(":")[0]
    foulWord = foulWord.strip()
    dictionary.append(foulWord.lower())

# create English stop words list
en_stop = get_stop_words('en')
cachedStopWords = stopwords.words("english")

#increasing the stop word list as the twitter data set is noisy
en_stop.extend(["demibestfans2016", "u", "rt", "t", "s", "updat", "channel", "de", "que", "la", "en", "eurekamag", "na", "sa", "ang", "keo", "ka", "lang", "le", "je", "est", "c", "pa", "j", "ik", "un", "et", "il", "wt", "fpjb", "fnfjb", "rbjb", "amp", "ini", "ada", "amant", "pushawardskathniel", "kathniel", "00", "05", "04", "15", "16", "14", "18", "aku", "niond", "da", "ich", "ero", "rtandfollow", "da", "ich", "und", "ist", "ero", "m", "da", "com", "em", "um", "meu", "na", "pra", "weather", "properti", "googl", "0mm", "co", "thttps", "https", "http", "n", "t", "u", "for", "us", "is:", "it.", "on", "i'll", "also", "of", "via", "follow", "mali", "rt", "got", "nowplay", "periscop", "stat", "replay", "katch", "biztip", "via", "radio", "commerci", "na", "sa", "ang", "ko", "ng", "mo", "aka", "ka", "ve", "ke", "kixmi", "capricorn", "tarnat", "today", "sagittariu", "tauru", "votingdevonne23", "ff", "0", "new", "go", "mm", "aku", "yang", "yg", "ni", "tak", "ada", "nak", "ya", "dutchschultz", "strictlybid", "lovat", "iheartaward", "bestfanarmi", "que", "la", "el", "en", "y", "lo", "es", "ain", "wit", "votingdevonne23", "giveyourheartdd", "amant", "ootd", "bandana", "updat", "get", "channel", "pushawardskathniel", "kathniel", "cbb", "uri", "santiago", "ain'", 'ain', "por", "para", "una", "der", "ein", "aja", "kamu", "sama", "untuk", "lagi", "ako", "kau", "dah", "dia", "kalu", "lah", "bilai", "apa", "lagi", "pushawardskathniels", "hahaha", "haha", "hahahaha", "bestfanarmy", "can", "don"])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_set=[]

# get the current path
path = os.getcwd()
# bring the path to speed
#path = path + "/ALLIN"
#path = path + "/NewDATAForExp/CSVs_NF_renamed"
#path = path + "/FILTERED_ALL"
path = path + "/TEMP"

#get the list of files
lst = os.listdir(path)
nameList = []
# Now take the files one by one and create one doc per user and add to list
startTime = datetime.now()

for fileName in lst:
    name = fileName.split(".")[0]
    nameList.append(name)
    temp = ""
    fileadd = path + "/"+fileName
    fileContent = open(fileadd)
    for line in fileContent:
        try:
            line = line.split(",")[2]
            # get rid of urls
            line = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',line)
            #to replace #words to just words 
            line = re.sub(r'#([^\s]+)', r'\1', line)
            #line = re.sub(r'#([^\s]+)', '', line)
            # userNames - no use- get rid
            line = re.sub('@[^\s]+','', line)
            # new lines were creating issues
            line = line.strip('\'"')
            # throwing exception because of unicode error - Fixed that
            line = unicode(line, "utf8")
            # prepare string
            temp = temp + str(line)
            #print temp
        except:
            pass
    doc_set.append(temp)

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    filtered_tokens = []
    frequency = defaultdict(int)
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    
    stopped_tokens = [i for i in tokens if not i in en_stop]

    
    #dict_tokens = [i for i in stopped_tokens if  i in dictionary]
    
    # stopwords from nltk package as well although there is not much difference 
    # dictionary approach is killing us time uwise
    #stopped_tokens = [i for i in tokens if not i in cachedStopWords]
    
    # stem tokens
    # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    #stemmed_tokens = [p_stemmer.stem(i) for i in dict_tokens]

    for element in stopped_tokens:
        if len(element) > 2:
            filtered_tokens.append(element)
    

    # how about trying stemming once all the two length words are out 
    stemmed_tokens = [p_stemmer.stem(i) for i in filtered_tokens]

    # now going to give a attempt try to correct the words - removing the stemmer for now problem: # spell correction gave us 2 min / document not a feasible option hence
    spellcorrected_tokens = []
    '''for element in stemmed_tokens:
        b = TextBlob(element)
        word = b.correct()
        spellcorrected_tokens.append(str(word))
    '''
    texts.append(stemmed_tokens)
    #texts.append(filtered_tokens)
    #texts.append(spellcorrected_tokens)

#if word has appeared just once in corpus - throw that out 
for text in texts:
    for token in text:
        frequency[token] += 1


texts = [[token for token in text if frequency[token] > 3] for text in texts]


# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)


#print type(dictionary)

#print dictionary

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=30)

print(ldamodel.print_topics(num_topics=3, num_words = 10))

ldamodel.save('./data/lda_twitter_6310user_100topics.model')

counter = 0

for item  in corpus:
    '''print "*************************************************************"
    print item'''
    print nameList[counter],
    counter = counter + 1
    print ldamodel[item]

print datetime.now() - startTime





[(0, u'0.179*video + 0.174*like + 0.014*soprano + 0.012*movi + 0.009*scene + 0.008*prank + 0.008*clip + 0.007*toni + 0.007*trailer + 0.006*offici'), (1, u'0.036*nigga + 0.030*lol + 0.024*shit + 0.022*bitch + 0.017*like + 0.016*just + 0.014*fuck + 0.010*dat + 0.010*ass + 0.009*know'), (2, u'0.070*book + 0.045*seller + 0.040*best + 0.021*amazon + 0.021*hot + 0.016*kindl + 0.016*know + 0.015*just + 0.011*love + 0.010*publish')]
SevenEasyWays [(2, 0.99997132468065919)]
SumMannShit [(1, 0.9999312469496654)]
classi4u [(2, 0.99964566145472233)]
UnkleAce [(0, 0.99995599955702952)]
0:00:07.373057


In [14]:
print corpus

[[(0, 7), (1, 36), (2, 23), (3, 5), (4, 5), (5, 3), (6, 8), (7, 5), (8, 5), (9, 11), (10, 4), (11, 12), (12, 27), (13, 9), (14, 10), (15, 8), (16, 3), (17, 167), (18, 100), (19, 5), (20, 7), (21, 4), (22, 33), (23, 1), (24, 6), (25, 10), (26, 2), (27, 42), (28, 12), (29, 1), (30, 7), (31, 5), (32, 9), (33, 56), (34, 2), (35, 46), (36, 2), (37, 1), (38, 1), (39, 4), (40, 5), (41, 34), (42, 9), (43, 6), (44, 16), (45, 8), (46, 13), (47, 13), (48, 8), (49, 33), (50, 68), (51, 11), (52, 17), (53, 21), (54, 10), (55, 1), (56, 4), (57, 24), (58, 1), (59, 6), (60, 7), (61, 3), (62, 10), (63, 33), (64, 146), (65, 7), (66, 19), (67, 1), (68, 6), (69, 1), (70, 9), (71, 2), (72, 5), (73, 5), (74, 28), (75, 2), (76, 2), (77, 1), (78, 9), (79, 11), (80, 33), (81, 5), (82, 41), (83, 275), (84, 4), (85, 2), (86, 86), (87, 4), (88, 3), (89, 18), (90, 6), (91, 20), (92, 6), (93, 28), (94, 45), (95, 4), (96, 1), (97, 35), (98, 321), (99, 36), (100, 37), (101, 6), (102, 25), (103, 15), (104, 17), (105, 1

In [17]:
path = os.getcwd()
path = path + "/TEST"

doc_test= []
#get the list of files
lst = os.listdir(path)
nameList = []
# Now take the files one by one and create one doc per user and add to list
startTime = datetime.now()

for fileName in lst:
    name = fileName.split(".")[0]
    nameList.append(name)
    temp = ""
    fileadd = path + "/"+fileName
    fileContent = open(fileadd)
    for line in fileContent:
        try:
            line = line.split(",")[2]
            # get rid of urls
            line = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',line)
            #to replace #words to just words 
            line = re.sub(r'#([^\s]+)', r'\1', line)
            #line = re.sub(r'#([^\s]+)', '', line)
            # userNames - no use- get rid
            line = re.sub('@[^\s]+','', line)
            # new lines were creating issues
            line = line.strip('\'"')
            # throwing exception because of unicode error - Fixed that
            line = unicode(line, "utf8")
            # prepare string
            temp = temp + str(line)
            #print temp
        except:
            pass
    doc_test.append(temp)

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_test:
    filtered_tokens = []
    frequency = defaultdict(int)
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    
    stopped_tokens = [i for i in tokens if not i in en_stop]

   
    for element in stopped_tokens:
        if len(element) > 2:
            filtered_tokens.append(element)
    

    # how about trying stemming once all the two length words are out 
    stemmed_tokens = [p_stemmer.stem(i) for i in filtered_tokens]

    texts.append(stemmed_tokens)
    

#if word has appeared just once in corpus - throw that out 
for text in texts:
    for token in text:
        frequency[token] += 1


texts = [[token for token in text if frequency[token] > 1] for text in texts]


# convert tokenized documents into a document-term matrix
corpus_test = [dictionary.doc2bow(text) for text in texts]


counter = 0

for item  in corpus_test:
    print nameList[counter],
    print ldamodel[item]
    counter = counter +1

MackNSweetJones [(0, 0.13342999404532668), (1, 0.83058558226689516), (2, 0.035984423687778148)]
ParsonsBrett2 [(0, 0.04746885090281195), (1, 0.55262743040106976), (2, 0.39990371869611829)]
